In [4]:
# Ensure that openai and python-dotenv are installed in the Jupyter Notebook environment
%pip install openai python-dotenv

# Import necessary libraries
import openai
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Upload the training file
response = openai.File.create(
    file=open("../data/training_data.jsonl", "rb"),
    purpose='fine-tune'
)
training_file_id = response['id']
print(f"Training file ID: {training_file_id}")

# Verify the file upload
file_info = openai.File.retrieve(training_file_id)
print(file_info)

# Fine-tune the model
fine_tune_response = openai.FineTune.create(
    training_file=training_file_id,
    model="curie"  # or "davinci", "babbage", etc.
)
fine_tune_id = fine_tune_response['id']
print(f"Fine-tuning job ID: {fine_tune_id}")

# Check the status of the fine-tuning job
status = openai.FineTune.retrieve(fine_tune_id)
print(status)

# Optionally, monitor the fine-tuning process
def check_fine_tune_status(fine_tune_id):
    status = openai.FineTune.retrieve(fine_tune_id)
    status_str = status['status']
    print(f"Fine-tuning status: {status_str}")
    return status_str

# Monitor the fine-tuning job until it's done
while True:
    status_str = check_fine_tune_status(fine_tune_id)
    if status_str not in ['pending', 'running']:
        break
    time.sleep(60)  # Wait for 60 seconds before checking the status again

# Use the fine-tuned model once the fine-tuning is complete
if status_str == 'succeeded':
    response = openai.Completion.create(
        model=fine_tune_id,  # Use the ID of the fine-tuned model
        prompt="Provide a lesson plan on Introduction to Addition and Subtraction suitable for students with Dyslexia and ADHD, for Kindergarten through Grade 2.",
        max_tokens=1000
    )

    print(response['choices'][0]['text'])
else:
    print("Fine-tuning did not succeed.")


Note: you may need to restart the kernel to use updated packages.


APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
